# Save & Load Model

In [1]:
import torch
import torch.onnx as onnx
import torchvision.models as models

#### Saving and Loading Model Weights
保存和加载模型权重
PyTorch模型将学习到的参数存储在称为的内部状态字典中state_dict。这些可以通过以下torch.save 方法持久化：

In [2]:
model = models.vgg16(pretrained=True) # pretrained (bool): If True, 则返回在ImageNet上预训练的模型
torch.save(model.state_dict(), 'model_weights.pth')

In [3]:
# 要加载模型权重，您需要首先创建相同模型的实例，然后使用load_state_dict()方法加载参数。
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
# 确保model.eval()在推断之前调用。
# 以将dropout和batch规范化层设置为评估模式。不这样做将产生不一致的推论结果。
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#### Saving and Loading Models with Shapes
加载模型权重时，我们需要首先实例化模型类，因为该类定义了网络的结构。

我们可能希望将此类的结构与模型一起保存，在这种情况下，我们可以将model（而不是model.state_dict()）传递给保存函数：

In [4]:
torch.save(model, 'model.pth')

In [5]:
#加载模型
model = torch.load('model.pth')

#### Exporting Model to ONNX
鉴于PyTorch执行图的动态性质，导出过程必须遍历执行图以生成持久化的ONNX模型。因此，应将适当大小的测试变量传递到导出例程中（在我们的示例中，我们将创建正确大小的虚拟零张量）：

In [7]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'model.onnx')